# Problem Set 2022/23 - Simon Hilding-Nørkjær (xsj415)


### Pre-loading packages and functions: 

In [1]:
import numpy as np                                     
from numpy.linalg import matrix_power                  
import matplotlib.pyplot as plt                                                        
from iminuit import Minuit                            
import sys                                            
from scipy import stats
import sympy as sp
from sympy import integrate

plt.style.use('classic')

sys.path.append('../External_Functions')
from ExternalFunctions import UnbinnedLH, BinnedLH, Chi2Regression
from ExternalFunctions import nice_string_output, add_text_to_ax   

## I - Distributions and probabilities:

### 1.1) Your friend tells you, that a bag contains 3 white, 6 black, and 7 grey marbles.

#### 1.1.1) If you take two random marbles without putting them back, what is the probability that at least one of them is white?

In [64]:
n_white , n_black , n_grey = 3 , 6 , 7
N = n_white + n_black + n_grey

#first draw
p_white = n_white/N
p_other = 1-p_white

#second draw
p_other_given_other = (n_black+n_grey-1)/(N-1)
p_atleast1white = 1 - p_other*p_other_given_other

print('Probability of at least one white marble =',p_atleast1white)  

Probability of at least one white marble = 0.35


#### 1.1.2) If you pick a marble, record its color, and then put it back 25 times independently, what is the probability of getting exactly 18 grey marbles? At least 18 grey marbles?

In [65]:
P_18grey_of_25 = stats.binom.pmf(18,25,p_grey)
P_atleast_18grey_of_25 = stats.binom.sf(17,25,p_grey)
print('Probability of 18 grey marbles out of 25 trials =',P_18grey_of_25)
print('Probability of at least 18 grey marbles out of 25 trials =',P_atleast_18grey_of_25)

Probability of 18 grey marbles out of 25 trials = 0.002953499094615582
Probability of at least 18 grey marbles out of 25 trials = 0.004039599792444158


#### 1.1.3) If you got 18 grey marbles out of 25 picks, would you trust your friend’s information?

In [66]:

# expected number of grey marbles
Expected_grey = 25*p_grey
# binomial standard deviation of grey marbles
sigma_grey = np.sqrt(25*p_grey*(1-p_grey))    
# z-score
z = (18-Expected_grey)/sigma_grey
# p-value
p = stats.norm.sf(z)


print('z-score =',z)
print('p-value =',p)
print('Conclusion: I would not trust the information as the p-value based on the z-score show is really low.')

z-score = 2.8473323633361787
p-value = 0.0022043653491208926
Conclusion: I would not trust the information as the p-value based on the z-score show is really low.


### 1.2) 
The lifetime L of a certain component is exponentially distributed: L(t) = 1/τ exp(−t/τ ).
If there is a 4% chance of this component lasting more than 500 hours, what is the value of τ ?

In [91]:
#solving the intgral of L(t) = 1/τ exp(−t/τ ) from 0 to 500 using sympy 

t, tau = sp.symbols('t tau', real=True)
L = 1/tau*sp.exp(-t/tau)
P_L500 = integrate(L,(t,500,sp.oo),conds='none')
Tau = sp.solve(P_L500-0.04,tau)   #solving for tau and because the integral is from 500 to infinity

print(r'The lifetime of the particle is {:.2f}, hours'.format(Tau[0]))

The lifetime of the particle is 155.33, hours


### 1.3) A radio telescope detects 241089 signals/day, based on a 9 week observation campaign.

#### 1.3.1) One hour, it receives 9487 signals. What is the chance of observing exactly this number?

In [3]:
detection_rate = 241089       #counts per day 
signal = 9487                 #signal in one hour

#probability of detecting the signal in one hour
P_signal = stats.poisson.pmf(signal,detection_rate/24)
print('Probability of detecting the signal in one hour: ',P_signal.round(12))


Probability of detecting the signal in one hour:  5.55e-10


#### 1.3.2) Is this observation extraordinary, based on an estimate of its general probability?

In [4]:
#expected number of signal in one hour
Expected_signal = detection_rate/24

#Poissonian standard deviation of signal in one hour
sigma_signal = np.sqrt(detection_rate/24)

#z-score
z = (abs(signal-Expected_signal))/sigma_signal

#p-value
p = stats.norm.sf(z)

print('z-score =',z)
print('p-value =',p)
print('Conclusion: Very unlikely observation based on p-value and z-score.')

z-score = 5.571124816057189
p-value = 1.2654995762494655e-08
Conclusion: Very unlikely observation based on p-value and z-score.


### 1.4 )  Shooting with a bow, you have 3% chance of hitting a certain target

#### 1.4.1) What distribution is the number of hits going to follow, given N shots?

In [45]:
#Binomial distribution with p=0.03 give N shots

#### 1.4.2) What is the probability that the first hit will come after 20 shots?

In [60]:
#probability of getting first hit within 20 shots
P_firsthit = (1-p)**20 * p
print(f'Probability of getting first hit after 20 shots given N shots: {P_firsthit:.3f}')

Probability of getting first hit after 20 shots given N shots: 0.016


#### 1.4.3) What is the probability that it will take more than 4000 shots to hit the target 100 times?

In [63]:
#probability of it taking more than 4000 shots to hit 100 times 
P_4000shots = stats.binom.sf(99,4000,p)
print('Probability of it taking more than 4000 shots to hit 100 times: ',P_4000shots.round(3))

Probability of it taking more than 4000 shots to hit 100 times:  0.974


## II - Error Propagation

### 2.1)  Let x = 1.92 ± 0.39 and y = 3.1 ± 1.3, and let z1 = y/x, and z2 = cos(x) · x/y

In [61]:
x =  1.92 
xerr = 0.39
y = 3.1 
yerr = 0.3

def z1(x,y):
    return 

#### 2.1.1)  What are the uncertainties of z1 and z2, if x and y are uncorrelated?

### 2.2) Five patients were given a drug to test if they slept longer (in hours). 
##### Their results were:+3.7, −1.2, −0.2, +0.7, +0.8. A Placebo group got the results: +1.5, −1.0, −0.7, +0.5, +0.1

In [89]:
drug_group = np.array([3.7, -1.2, -0.2, 0.7, 0.8])
placebo_group = np.array([1.5, -1.0, -0.7, 0.5, 0.1])

#### 2.2.1) Estimate the mean, standard deviation, and the uncertainty on the mean for drug group

In [108]:
#mean , std and error on the mean of drug group
mean_drug = np.mean(drug_group)
std_drug = np.std(drug_group, ddof=1)
err_drug = std_drug/np.sqrt(len(drug_group))

#mean , std and error on the mean of placebo group
mean_placebo = np.mean(placebo_group)
std_placebo = np.std(placebo_group, ddof=1)
err_placebo = std_placebo/np.sqrt(len(placebo_group))


# print the results
print('Drug group: Mean = {:.2f} +/- {:.2f}, STD of distribution = {:.2f}'.format(mean_drug,err_drug,std_drug))
print('Placebo group: Mean = {:.2f} +/- {:.2f}, STD of distribution = {:.2f}'.format(mean_placebo,err_placebo,std_placebo))

Drug group: Mean = 0.76 +/- 0.82, STD of distribution = 1.83
Placebo group: Mean = 0.08 +/- 0.45, STD of distribution = 1.00


#### 2.2.2) What is the probability that the drug group slept longer than the placebo group?

As the means and std's for the two samples are estimated we have to use the students t test in order to test the null hypothesis that the true sample means are equal. 

In [128]:
#calculate the student t-test for the means of two independent samples of scores
t, p_ttest = stats.ttest_ind(drug_group, placebo_group, equal_var=False, alternative='greater')

print(f't-statistic = {t:.2f}')
print(f'p-value = {p_ttest:.2f}')

# comparing with kolmogorov-smirnov test
D, p_ks = stats.ks_2samp(drug_group, placebo_group, alternative='greater')

print(f'D-statistic = {D:.2f}')
print(f'p-value = {p_ks:.2f}')

print('Conclusion: The p-value from the t-test is smaller than the p-value from the KS test. Because the means and the STDs are estimated the t-test should perform better, and therefore it is more likely to reject the null hypothesis.')

t-statistic = 0.73
p-value = 0.25
D-statistic = 0.20
p-value = 0.83
Conclusion: The p-value from the t-test is smaller than the p-value from the KS test. Because the means and the STDs are estimated the t-test should perform better, and therefore it is more likely to reject the null hypothesis.


## III - Simulation / Monte Carlo:

### 3.1) Assume $f(x) = Cx^asin(πx)$ for x ∈ [0, 1] and a = 3 is a theoretical distribution.

#### 3.1.1) By what method(s) would you generate random numbers according to f (x)?




#### 3.1.2) Determine (possibly numerically) the value of C for f (x) to be normalised.


#### 3.1.3) Fit a histogram with values from f (x) and determine how many measurements (i.e. values of x) you need in an experiment to determine the value of a with 1% precision.